In [259]:
with open('libraries.py') as f:
    code = f.read()
exec(code)

with open('functions.py') as f:
    code = f.read()
exec(code)

In [260]:
pd.reset_option('all')
pd.set_option('display.max_rows', 100)

In [261]:
# determine user
user = getpass.getuser()
if user == 'peymansh':
    main_folder_path = '/Users/peymansh/Dropbox (MIT)/Research/AI and Occupations/ai-exposure'
    data_path = f'{main_folder_path}/output'

In [262]:
onet_data_path = f'{data_path}/data/onet_occupations_yearly.csv'

# list of occupations to create DAGs for
occupation_list = ['travelAgents', 'insuranceUnderwriters', 'pileDriverOperators', 
                   'dredgeOperators', 'gradersAndSortersForAgriculturalProducts', 'reinforcingIronAndRebarWorkers',
                   'insuranceAppraisersForAutoDamage', 'floorSandersAndFinishers', 'dataEntryKeyer', 
                   'athletesAndSportsCompetitors', 'audiovisualEquipmentInstallerAndRepairers', 'hearingAidSpecialists', 
                   'personalCareAides', 'proofreadersAndCopyMarkers', 'chiropractors', 
                   'shippingReceivingAndInventoryClerks', 'cooksShortOrder', 'orthodontists',
                   'subwayAndStreetcarOperators', 'packersAndPackagersHand', 'hoistAndWinchOperators', 
                   'forgingMachineSettersOperatorsAndTenders', 'avionicsTechnicians', 'dishwashers', 
                   'dispatchersExceptPoliceFireAndAmbulance', 'familyMedicinePhysicians', 'MachineFeedersAndOffbearers'
                   ]

occupation = 'travelAgents'
occupation = 'insuranceUnderwriters'
occupation = 'pileDriverOperators'

# Generate occupation-specific strings
GPT_input_occupation, plot_title_occupation, occupation_code, occupation_folder = pick_occupation(occupation)

In [263]:
# set alpha as AI quality metric
n = 100
epsilon = 1e-8
alpha_list = np.linspace(epsilon, 1-epsilon, n).tolist()

### Initialize input-output paths

In [264]:
# Manual DAG
input_path = f'{occupation_folder}/{occupation}_M_DAG_df.csv'
output_path = f'{occupation_folder}/{occupation}_costMin_M.csv'

# # First Last Task DAG
# input_path = f'{occupation_folder}/{occupation}_FLT_GPT_DAG_df.csv'
# output_path = f'{occupation_folder}/{occupation}_costMin_FLT.csv'

# # Conditioned First Last Task DAG
# input_path = f'{occupation_folder}/{occupation}_CFLT_GPT_DAG_df.csv'
# output_path = f'{occupation_folder}/{occupation}_costMin_CFLT.csv'

# # Partitioned DAG
# input_path = f'{occupation_folder}/{occupation}_P_GPT_DAG_df.csv'
# output_path = f'{occupation_folder}/{occupation}_costMin_P.csv'

# # Conditioned Partitioned DAG
# input_path = f'{occupation_folder}/{occupation}_CP_GPT_DAG_df.csv'
# output_path = f'{occupation_folder}/{occupation}_costMin_CP.csv'

In [265]:
# read DAG
dag_df = pd.read_csv(input_path)

# remove edges if comment column labeled with "TriangleRemovedFlag" (edge is there for plotting purposes and is not part of the actual DAG)
if 'comment' in dag_df.columns:
    dag_df = dag_df[~dag_df['comment'].str.endswith('TriangleRemovedFlag')]

# get task stats
tasks_stats = pd.read_csv(f'{occupation_folder}/{occupation}_taskStats.csv')
tasks_stats

# print stats
#tasks_stats.iloc[:,1:].sum()
dag_df

,source,target
0,Move hand and foot levers of hoisting equipmen...,Drive pilings to provide support for buildings...
1,Conduct pre-operational checks on equipment to...,"Clean, lubricate, and refill equipment."
2,Drive pilings to provide support for buildings...,"""Target"""
3,Move levers and turn valves to activate power ...,Drive pilings to provide support for buildings...
4,"Clean, lubricate, and refill equipment.",Move hand and foot levers of hoisting equipmen...
5,"Clean, lubricate, and refill equipment.",Drive pilings to provide support for buildings...
6,"Clean, lubricate, and refill equipment.",Move levers and turn valves to activate power ...


In [266]:
# extract list of tasks and create a dictionary for indexing tasks
tasks_list = tasks_stats['task'].unique()
tasks_dict = {i: node for i, node in enumerate(tasks_list, start=0)}

# create numpy array of adjacency matrix
adjacency_matrix = np.zeros((len(tasks_list), len(tasks_list)), dtype=int)
aux_dict = {value: key for key, value in tasks_dict.items()}
for _, row in dag_df.iterrows():
    source_index = aux_dict[row['source']]
    target_index = aux_dict[row['target']]
    adjacency_matrix[source_index, target_index] = 1

tasks_dict

{0: 'Move hand and foot levers of hoisting equipment to position piling leads, hoist piling into leads, and position hammers over pilings.',
 1: 'Conduct pre-operational checks on equipment to ensure proper functioning.',
 2: 'Drive pilings to provide support for buildings or other structures, using heavy equipment with a pile driver head.',
 3: 'Move levers and turn valves to activate power hammers, or to raise and lower drophammers that drive piles to required depths.',
 4: 'Clean, lubricate, and refill equipment.',
 5: '"Target"'}

In [267]:
def find_neighbors(adjacency_matrix):
    n = adjacency_matrix.shape[0]  # Number of nodes
    neighbors_dict = {}

    for node in range(n):
        neighbors = []
        for neighbor in range(n):
            if adjacency_matrix[node, neighbor] != 0:  # Check for an edge from node to neighbor
                neighbors.append(neighbor)
        neighbors_dict[node] = neighbors

    # add node itself to set of its neighbors
    for key in neighbors_dict.keys():
        neighbors_dict[key].append(key)

    return neighbors_dict

In [268]:
neighbors = find_neighbors(adjacency_matrix[:-1,:-1])
neighbors

{0: [2, 0], 1: [4, 1], 2: [2], 3: [2, 3], 4: [0, 2, 3, 4]}

In [269]:
from itertools import combinations

def get_node_inclusive_partitions(arr, element):
    # Ensure the element is in the list
    if element not in arr:
        return []

    # Remove the specific element from the list
    arr.remove(element)
    
    # Generate all subsets of the remaining elements
    subsets = []
    for r in range(len(arr) + 1):
        subsets.extend(combinations(arr, r))
    
    # Add the specific element back to each subset
    subsets_with_element = []
    for subset in subsets:
        subsets_with_element.append((element,) + subset)

    subsets_with_element = [list(subset) for subset in subsets_with_element]
    return subsets_with_element

# Example usage
my_list = [0, 1, 2]
specific_element = 0
result = get_node_inclusive_partitions(my_list, specific_element)

print(result)

[[0], [0, 1], [0, 2], [0, 1, 2]]


In [270]:
def keep_my_unique_partitions(my_list):
    unique_values_tuple = set(tuple(sorted(inner_list)) for inner_list in my_list)
    unique_values_lists = [list(inner_tuple) for inner_tuple in unique_values_tuple]
    unique_values_lists.sort(key=len)
    return unique_values_lists



def keep_unique_partitions(dict):
    for key, value in dict.items():
        if len(value) > 0:
            # Normalize each inner list by sorting and then convert to tuple for set operations
            unique_values_lists = set(tuple(sorted(inner_list)) for inner_list in value)
            # Convert back to list of lists
            unique_values_lists = [list(inner_tuple) for inner_tuple in unique_values_lists]
            # Sort by the length of the inner lists
            unique_values_lists.sort(key=len)
            # Update the dictionary
            dict[key] = unique_values_lists
    return dict

In [271]:
def get_valid_DAG_subsets(adjacency_matrix):

    # get neighbors of each task (excluding Target node)
    tasks_neighbors = find_neighbors(adjacency_matrix[:-1,:-1])
    print(tasks_neighbors)

    # get list of tasks with no outgoing edges (i.e., last tasks)
    last_tasks_list = [key for key, value in tasks_neighbors.items() if len(value) == 1]
    print(last_tasks_list)


    # get number of non-Target nodes
    n = adjacency_matrix.shape[0] - 1

    # initialize dict for valid subsets of nodes (and also partitions) to act as memory
    memory_dict = {key: [] for key in range(n)}
    

    # initialize dictionary for valid subsets origniating from each node
    valid_subsets_dict = {}

    def valid_subsets_recursive(partition):
        print(f'partition {partition}')
        # if partition contains single node, either terminal node or non-terminal node
        if len(partition) == 1:

            # if node already in memory return corresponding value
            if len(memory_dict[partition[0]]) > 0:
                print(f'node {partition[0]} already calculated:', memory_dict[partition[0]])
                return memory_dict[partition[0]]
            

            # if node not in memory, check if terminal node or not
            # if terminal node (i.e., has no neighbors except itself), return node itself + empty list
            if len(tasks_neighbors[partition[0]]) == 1:
                print(f'terminal node subset:', [[partition[0]], []])
                # update memory
                memory_dict[partition[0]] = [[partition[0]], []]
                return [[partition[0]], []]
            
            # if non-terminal node, run recursive function on node's neighbors
            else:
                neighbors_list = tasks_neighbors[partition[0]].copy() # make sure to copy, otherwise will change the original list

                # remove node itself from neighbors list to run a for loop only on neighbors
                neighbors_list.remove(partition[0])

                # initialize list for valid subsets in all neighbors
                non_terminal_node_valid_subsets = []
                for neighbor in neighbors_list:
                    neighbor_valid_subsets = valid_subsets_recursive([neighbor])

                    # add non-terminal node to neighbor's valid subsets
                    neighbor_valid_subsets = [subset + [partition[0]] for subset in neighbor_valid_subsets]

                    #print(f'neighbor valid subsets:', neighbor_valid_subsets)

                    # append neighbor valid subsets to non-terminal node valid subsets
                    non_terminal_node_valid_subsets.extend(neighbor_valid_subsets)

                # add non-terminal node itself as singleton to set of valid subsets
                non_terminal_node_valid_subsets.append([partition[0]])
                non_terminal_node_valid_subsets = keep_my_unique_partitions(non_terminal_node_valid_subsets)

                print(f'node {partition[0]} is non-terminal node w/ valid subsets:', non_terminal_node_valid_subsets)
                # update memory
                memory_dict[partition[0]] = non_terminal_node_valid_subsets
                return non_terminal_node_valid_subsets
                
        
        # if partition contains more than one node
        else:
            
            # if partition already in memory return corresponding value
            try:
                if len(memory_dict[tuple(partition)]) > 0:
                    print(f'partition {partition} already calculated:', memory_dict[tuple(partition)])
                    return memory_dict[tuple(partition)]
            
            
            # if partition not in memory, get valid subsets of partition
            except KeyError:
                neighbors_list =[]
                for node in partition:
                    neighbors_list.extend(tasks_neighbors[node])

                # remove duplicates and partition nodes to run a for loop only on neighbors
                neighbors_list = list(set(neighbors_list) - set(partition))
                if len(neighbors_list) == 0:
                    return []
                
                # initialize list for valid subsets in all neighbors
                partition_valid_subsets = []
                for neighbor in neighbors_list:
                    neighbor_valid_subsets = valid_subsets_recursive([neighbor])

                    # add partition nodes to neighbor's valid subsets
                    neighbor_valid_subsets = [subset + partition for subset in neighbor_valid_subsets]

                    print(f'neighbor valid subsets:', neighbor_valid_subsets)

                    # append neighbor valid subsets to non-terminal node valid subsets
                    partition_valid_subsets.extend(neighbor_valid_subsets)

                # add partition itself to set of valid subsets
                partition_valid_subsets.append(partition)
                partition_valid_subsets = keep_my_unique_partitions(partition_valid_subsets)

                print(f'partition valid subsets:', partition_valid_subsets)
                # update memory
                memory_dict[tuple(partition)] = partition_valid_subsets
                return partition_valid_subsets   


    for node in range(n):
        # compute potential partitions including node
        neighbors_list = tasks_neighbors[node].copy() # make sure to copy, otherwise will change the original list
        print(f'\n\n\nneighbors of node {node}:', neighbors_list)
        node_partitions = get_node_inclusive_partitions(neighbors_list, node)
        print(f'partitions of {node}:', node_partitions, '\n')
        
        # run for loop on each partition of node
        node_valid_subsets = []
        for partition in node_partitions:
            partition_valid_subsets = valid_subsets_recursive(partition)
            print(f'\noutput of node {node} partition {partition}:', partition_valid_subsets)
            node_valid_subsets.extend(partition_valid_subsets)
            print(f'valid subsets of node {node} to this point:', node_valid_subsets, '\n')

        node_valid_subsets = keep_my_unique_partitions(node_valid_subsets)
        valid_subsets_dict[node] = node_valid_subsets


    # # modify valid subsets of last tasks
    # for last_task in last_tasks_list:
    #     last_task_neighbors = [i for i in range(n) if adjacency_matrix[i, last_task] != 0]
    #     last_task_neighbors.append(last_task)
        
    #     print(f'last task {last_task} neighbors:', last_task_neighbors)
    #     node_partitions = get_node_inclusive_partitions(last_task_neighbors, last_task)
    #     print(f'partitions of last task {last_task}:', node_partitions, '\n')

    #     # update valid_subsets_dict for last task
    #     # exclude partitions with only two nodes as they already appear in parent node partitions
    #     valid_subsets_dict[last_task] = [partition for partition in node_partitions if len(partition) != 2]

    # modify valid subsets of last tasks
    for last_task in last_tasks_list:
        valid_subsets_dict[last_task] = [[last_task]]


    #print(memory_dict)
    return valid_subsets_dict

In [272]:
tasks_dict

{0: 'Move hand and foot levers of hoisting equipment to position piling leads, hoist piling into leads, and position hammers over pilings.',
 1: 'Conduct pre-operational checks on equipment to ensure proper functioning.',
 2: 'Drive pilings to provide support for buildings or other structures, using heavy equipment with a pile driver head.',
 3: 'Move levers and turn valves to activate power hammers, or to raise and lower drophammers that drive piles to required depths.',
 4: 'Clean, lubricate, and refill equipment.',
 5: '"Target"'}

In [273]:
valid_subsets_dict = get_valid_DAG_subsets(adjacency_matrix)
valid_subsets_dict

{0: [2, 0], 1: [4, 1], 2: [2], 3: [2, 3], 4: [0, 2, 3, 4]}
[2]



neighbors of node 0: [2, 0]
partitions of 0: [[0], [0, 2]] 

partition [0]
partition [2]
terminal node subset: [[2], []]
node 0 is non-terminal node w/ valid subsets: [[0], [0, 2]]

output of node 0 partition [0]: [[0], [0, 2]]
valid subsets of node 0 to this point: [[0], [0, 2]] 

partition [0, 2]

output of node 0 partition [0, 2]: []
valid subsets of node 0 to this point: [[0], [0, 2]] 




neighbors of node 1: [4, 1]
partitions of 1: [[1], [1, 4]] 

partition [1]
partition [4]
partition [0]
node 0 already calculated: [[0], [0, 2]]
partition [2]
node 2 already calculated: [[2], []]
partition [3]
partition [2]
node 2 already calculated: [[2], []]
node 3 is non-terminal node w/ valid subsets: [[3], [2, 3]]
node 4 is non-terminal node w/ valid subsets: [[4], [2, 4], [0, 4], [3, 4], [0, 2, 4], [2, 3, 4]]
node 1 is non-terminal node w/ valid subsets: [[1], [1, 4], [0, 1, 4], [1, 3, 4], [1, 2, 4], [0, 1, 2, 4], [1, 2, 3, 4]

{0: [[0], [0, 2]],
 1: [[1], [1, 4], [0, 1, 4], [1, 3, 4], [1, 2, 4], [0, 1, 2, 4], [1, 2, 3, 4]],
 2: [[2]],
 3: [[3], [2, 3]],
 4: [[4],
  [2, 4],
  [0, 4],
  [3, 4],
  [0, 2, 4],
  [0, 3, 4],
  [2, 3, 4],
  [2, 2, 3, 4],
  [0, 2, 3, 4],
  [0, 2, 2, 4],
  [0, 2, 2, 3, 4]]}

In [274]:
# example_adjacency_matrix = np.array([[0, 1, 1, 0, 0],
#                                      [0, 0, 0, 1, 0],
#                                      [0, 0, 0, 1, 0],
#                                      [0, 0, 0, 0, 0],
#                                      [0, 0, 0, 0, 0]])
# adjacency_matrix = example_adjacency_matrix.copy()

# valid_subsets_dict = get_valid_DAG_subsets(adjacency_matrix)
# valid_subsets_dict

In [275]:
valid_subsets_dict = keep_unique_partitions(valid_subsets_dict)
valid_subsets_dict

{0: [[0], [0, 2]],
 1: [[1], [1, 4], [0, 1, 4], [1, 3, 4], [1, 2, 4], [0, 1, 2, 4], [1, 2, 3, 4]],
 2: [[2]],
 3: [[3], [2, 3]],
 4: [[4],
  [2, 4],
  [0, 4],
  [3, 4],
  [0, 2, 4],
  [0, 3, 4],
  [2, 3, 4],
  [2, 2, 3, 4],
  [0, 2, 3, 4],
  [0, 2, 2, 4],
  [0, 2, 2, 3, 4]]}

In [276]:
def is_combination_valid(combination, n):
    # Flatten list of combination
    covered_tasks_list = [element for sublist in combination for element in sublist]
    
    # Create a set of the flattened list
    covered_tasks_set = set(covered_tasks_list)
    
    # Check if the flattened set has exactly n elements and contains all elements from 0 to n-1
    if len(covered_tasks_list) == n and covered_tasks_set == set(range(n)):
        return True
    else:
        return False


def generate_combinations(valid_subsets_dict, current_key=0, current_combination=None, result=None):
    if current_combination is None:
        current_combination = []
    if result is None:
        result = []

    # Base case: if convered all tasks add current combination to the result list
    if is_combination_valid(current_combination, len(valid_subsets_dict)):
        print(f'********found valid combination: {current_combination}********')
        result.append(current_combination)
        return result

    # Recursive case: iterate through the list of lists at the current key
    for subset in valid_subsets_dict[current_key]:
        # Create a new combination including the current subset
        new_combination = current_combination + [subset]
        new_combination_flattened = [element for sublist in new_combination for element in sublist]

        # Check which nodes are NOT covered by the new combination. Only need to process these nodes next
        uncovered_nodes = list(set(range(len(valid_subsets_dict))) - set(new_combination_flattened))
        print(f'new combination: {new_combination}\n uncovered nodes: {uncovered_nodes}')
        #uncovered_nodes_valid_subsets_dict = {key: value for key, value in valid_subsets_dict.items() if key in uncovered_nodes}

        if len(uncovered_nodes) == 0:
            if is_combination_valid(new_combination, len(valid_subsets_dict)):
                print(f'********found valid combination: {new_combination}********\n')
                result.append(new_combination)
                return result
        else:
            # Recursively call the function to process the next key
            for nex_key in uncovered_nodes:
                generate_combinations(valid_subsets_dict, nex_key, new_combination, result)

    
    return result

In [277]:
combinations = []
for initial_node in range(len(valid_subsets_dict)):
    combinations.extend(generate_combinations(valid_subsets_dict, initial_node))

new combination: [[0]]
 uncovered nodes: [1, 2, 3, 4]
new combination: [[0], [1]]
 uncovered nodes: [2, 3, 4]
new combination: [[0], [1], [2]]
 uncovered nodes: [3, 4]
new combination: [[0], [1], [2], [3]]
 uncovered nodes: [4]
new combination: [[0], [1], [2], [3], [4]]
 uncovered nodes: []
********found valid combination: [[0], [1], [2], [3], [4]]********

new combination: [[0], [1], [2], [2, 3]]
 uncovered nodes: [4]
new combination: [[0], [1], [2], [2, 3], [4]]
 uncovered nodes: []
new combination: [[0], [1], [2], [2, 3], [2, 4]]
 uncovered nodes: []
new combination: [[0], [1], [2], [2, 3], [0, 4]]
 uncovered nodes: []
new combination: [[0], [1], [2], [2, 3], [3, 4]]
 uncovered nodes: []
new combination: [[0], [1], [2], [2, 3], [0, 2, 4]]
 uncovered nodes: []
new combination: [[0], [1], [2], [2, 3], [0, 3, 4]]
 uncovered nodes: []
new combination: [[0], [1], [2], [2, 3], [2, 3, 4]]
 uncovered nodes: []
new combination: [[0], [1], [2], [2, 3], [2, 2, 3, 4]]
 uncovered nodes: []
new c

In [278]:
print(len(combinations))
combinations

334


[[[0], [1], [2], [3], [4]],
 [[0], [1], [2], [4], [3]],
 [[0], [1], [2], [3, 4]],
 [[0], [1], [3], [2], [4]],
 [[0], [1], [3], [4], [2]],
 [[0], [1], [3], [2, 4]],
 [[0], [1], [2, 3], [4]],
 [[0], [1], [4], [2], [3]],
 [[0], [1], [4], [3], [2]],
 [[0], [1], [4], [2, 3]],
 [[0], [1], [2, 4], [3]],
 [[0], [1], [3, 4], [2]],
 [[0], [1], [2, 3, 4]],
 [[0], [1, 4], [2], [3]],
 [[0], [1, 4], [3], [2]],
 [[0], [1, 4], [2, 3]],
 [[0], [1, 3, 4], [2]],
 [[0], [1, 2, 4], [3]],
 [[0], [1, 2, 3, 4]],
 [[0], [2], [1], [3], [4]],
 [[0], [2], [1], [4], [3]],
 [[0], [2], [1], [3, 4]],
 [[0], [2], [1, 4], [3]],
 [[0], [2], [1, 3, 4]],
 [[0], [2], [3], [1], [4]],
 [[0], [2], [3], [1, 4]],
 [[0], [2], [3], [4], [1]],
 [[0], [2], [4], [1], [3]],
 [[0], [2], [4], [3], [1]],
 [[0], [2], [3, 4], [1]],
 [[0], [3], [1], [2], [4]],
 [[0], [3], [1], [4], [2]],
 [[0], [3], [1], [2, 4]],
 [[0], [3], [1, 4], [2]],
 [[0], [3], [1, 2, 4]],
 [[0], [3], [2], [1], [4]],
 [[0], [3], [2], [1, 4]],
 [[0], [3], [2], [4], [1

In [279]:
def normalize_sublist(sublist):
    # Sort the elements within each inner list and then sort the entire sublist
    return tuple(sorted(tuple(sorted(inner)) for inner in sublist))

def unique_lists(input_list):
    seen = set()
    unique_combinations = []

    for sublist in input_list:
        normalized = normalize_sublist(sublist)
        if normalized not in seen:
            seen.add(normalized)
            unique_combinations.append(sublist)

    return unique_combinations

unique_combinations = unique_lists(combinations)
print(len(unique_combinations))
unique_combinations

22


[[[0], [1], [2], [3], [4]],
 [[0], [1], [2], [3, 4]],
 [[0], [1], [3], [2, 4]],
 [[0], [1], [2, 3], [4]],
 [[0], [1], [2, 3, 4]],
 [[0], [1, 4], [2], [3]],
 [[0], [1, 4], [2, 3]],
 [[0], [1, 3, 4], [2]],
 [[0], [1, 2, 4], [3]],
 [[0], [1, 2, 3, 4]],
 [[0, 2], [1], [3], [4]],
 [[0, 2], [1], [3, 4]],
 [[0, 2], [1, 4], [3]],
 [[0, 2], [1, 3, 4]],
 [[1], [2], [3], [0, 4]],
 [[1], [2], [0, 3, 4]],
 [[1], [3], [0, 2, 4]],
 [[1], [2, 3], [0, 4]],
 [[1], [0, 2, 3, 4]],
 [[0, 1, 4], [2], [3]],
 [[0, 1, 4], [2, 3]],
 [[0, 1, 2, 4], [3]]]

### Generate all possible partition schemes for the set of tasks (ignoring structre of the DAG)

In [280]:
from itertools import combinations

def partitions(set_):
    if not set_:
        yield []
        return
    for i in range(1, len(set_) + 1):
        for part in combinations(set_, i):
            remaining = set(set_) - set(part)
            if not remaining:
                yield [list(part)]
            else:
                for b in partitions(list(remaining)):
                    yield [list(part)] + b

def generate_unique_partitions(numbers):
    all_partitions = set()
    for partition in partitions(numbers):
        # Create a frozenset of frozensets to make each partition hashable and order-independent
        partition_set = frozenset(frozenset(part) for part in partition)
        all_partitions.add(partition_set)
    
    # Convert the frozensets back to lists for the final output
    unique_partitions = [list(map(list, partition)) for partition in all_partitions]

    # Sort elements
    unique_partitions = sorted([sorted(x) for x in unique_partitions], key=len)
    return unique_partitions

In [281]:
# Generate list of numbers for non-"Target" tasks in occupation
tasks_list_numbers = list(range(len(valid_subsets_dict)))

# Generate all possible partitioning schemes
all_partitions = generate_unique_partitions(tasks_list_numbers)


### Check if partition scheme is "valid" (i.e., if its non-singleton partitions are a connected graph)

In [282]:
def is_connected(matrix):
    # Number of nodes in the matrix
    num_nodes = matrix.shape[0]
    
    # Visited array to keep track of visited nodes
    visited = np.zeros(num_nodes, dtype=bool)
    
    # Helper function to perform DFS
    def dfs(node):
        visited[node] = True
        # Visit all the neighbors of the current node
        for neighbor in range(num_nodes):
            if matrix[node, neighbor] == 1 and not visited[neighbor]:
                dfs(neighbor)
            elif matrix[neighbor, node] == 1 and not visited[neighbor]:
                dfs(neighbor)
    
    # Start DFS from the first node (node 0)
    dfs(0)
    
    # If all nodes are visited, the matrix is connected
    return np.all(visited)


def validate_partition_using_connectedness(adjacency_matrix, tasks_list):
    # Return valid if Singleton
    if len(tasks_list) == 1:
        return True
    # Check if partition forms connected graph
    else:
        # Subset original adjacency matrix
        subset_matrix = adjacency_matrix[np.ix_(tasks_list, tasks_list)]

        # check if subset matrix is a connected graph
        subset_matrix_connected = is_connected(subset_matrix)

        # return true if connected and false otherwise
        return subset_matrix_connected

In [283]:
# Get valid partitioning schemes from all possible partitions to cut computation load
valid_partitions = []
for scheme in all_partitions:
    # Set valid partitions count to 0
    valid_partition_count = 0
    for partition in scheme:
        valid_partition = validate_partition_using_connectedness(adjacency_matrix, partition)
        if valid_partition:
            valid_partition_count += 1
    
    # If number of valid partitions within a partition scheme is equal to 
    # number of partitions in partition scheme then partition scheme is valid
    if valid_partition_count == len(scheme):
        valid_partitions.append(scheme)

# Print stats
print(f'Number of all possible partitioning schemes: {len(all_partitions)}')
print(f'Number of valid partitioning schemes given DAG structure: {len(valid_partitions)}')

for partition in valid_partitions:
    print(partition)

Number of all possible partitioning schemes: 52
Number of valid partitioning schemes given DAG structure: 26
[[0, 1, 2, 3, 4]]
[[0, 2, 3, 4], [1]]
[[0, 2], [1, 3, 4]]
[[0, 1, 2, 4], [3]]
[[0, 2, 3], [1, 4]]
[[0, 1, 4], [2, 3]]
[[0, 1, 3, 4], [2]]
[[0], [1, 2, 3, 4]]
[[0, 2, 3], [1], [4]]
[[0, 2], [1, 4], [3]]
[[0, 3, 4], [1], [2]]
[[0, 1, 4], [2], [3]]
[[0, 2, 4], [1], [3]]
[[0], [1, 2, 4], [3]]
[[0, 4], [1], [2, 3]]
[[0, 2], [1], [3, 4]]
[[0], [1, 3, 4], [2]]
[[0], [1, 4], [2, 3]]
[[0], [1], [2, 3, 4]]
[[0, 2], [1], [3], [4]]
[[0], [1], [2], [3, 4]]
[[0, 4], [1], [2], [3]]
[[0], [1], [2, 3], [4]]
[[0], [1], [2, 4], [3]]
[[0], [1, 4], [2], [3]]
[[0], [1], [2], [3], [4]]


## Missing in "Smart" method

In [284]:
def normalize_sublist(sublist):
    # Sort the elements within each inner list and then sort the entire sublist
    return tuple(sorted(tuple(sorted(inner)) for inner in sublist))

def list_difference(list1, list2):
    # Normalize both lists
    normalized_list1 = {normalize_sublist(sublist) for sublist in list1}
    normalized_list2 = {normalize_sublist(sublist) for sublist in list2}
    
    # Find the difference
    difference = normalized_list1 - normalized_list2
    
    # Convert the normalized tuples back to the original list format
    difference_list = []
    for norm_sublist in difference:
        original_sublist = [list(inner) for inner in norm_sublist]
        difference_list.append(original_sublist)
    
    return difference_list

result = list_difference(valid_partitions, unique_combinations)
print(len(result))
for case in result:
    print(case)

4
[[0, 1, 2, 3, 4]]
[[0, 2, 3], [1, 4]]
[[0, 1, 3, 4], [2]]
[[0, 2, 3], [1], [4]]
